In [1]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
import csv
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

def predict_clossing_price(start_year, start_month, start_day, code):
    
    start = datetime.datetime(start_year,start_month,start_day)
    end = datetime.date.today()
    stockData = data.DataReader(code , 'yahoo',start,end)

    def get_rsi(sdata,m,mem):
        neg = 0
        pos = 0
        RS = 0
        RSI = 100

        upcloses = 0
        downcloses = 0

        n = m
        k = m-1

        for p in range(mem):
           diff = sdata[n,3] - sdata[k,3]
           if (diff>=0):
            upcloses = upcloses + diff
            pos = pos+1
           else:
            downcloses = downcloses + diff
            neg = neg+1

           n = n-1
           k = k-1

        downcloses = -downcloses
        if(neg == 0):
            return 100
        else:
            RS = (upcloses*neg)/(downcloses*pos)


        RSI = 100 - (100/(1+RS))

        return RSI


    def get_mfi(sdata,m,mem):
        neg = 0
        pos = 0
        MFR = 0
        MFI = 100

        pmflow = 0
        nmflow = 0

        n = m
        k = m-1

        for p in range(mem):

           typ_pricec = (sdata[n,0] + sdata[n,1] + sdata[n,3])/3
           typ_pricep = (sdata[k,0] + sdata[k,1] + sdata[k,3])/3

          # print(typ_price,sdata[n,0],sdata[n,1],sdata[n,3])

           if (typ_pricec>=typ_pricep):
             pmflow = pmflow + ((sdata[n,4])*(typ_pricec))
             pos = pos+1
           else:
             nmflow = nmflow + ((sdata[n,4])*(typ_pricec))
             neg = neg+1

           n = n-1
           k = k-1

        if(neg == 0):
            return 100
        else:
            MFR = pmflow/nmflow

        MFI = 100 - (100/(1+MFR))

        return MFI

    def get_ema(sdata,m,mem,EMAp):

       EMA = sdata[m,3]*(2/(1+mem)) + (1-(2/(1+mem)))*EMAp 
       #print("EMA",EMA)

       return EMA

    def get_so(sdata,m,mem):

       SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100
       #print("SO",SO)
       return SO


    memory = 14
    sdat2 = stockData.reset_index()
    del sdat2["Date"]
    del sdat2["Adj Close"]
    sdat3 = np.array(sdat2,dtype=np.float32)

    #print(sdat3[0])

    df1 = pd.DataFrame(columns=['Open','High','Low','Close','Volume','RSI','MFI','EMA','SO','CloseNext'])

    df2 = pd.DataFrame(columns=['Close','RSI','MFI','EMA','SO','CloseNext'])

    arr = np.array(df1.values)

    #print("Printing j\n")

    EMAp = 0
    acc = 0

    for i in range(memory):
        acc = acc + sdat3[i,3]

    EMAp = acc / memory    

    #for i in range(len(sdat3) -memory):
    for i in range(len(sdat3)-1 -memory):
        j = i + memory

        RSI = get_rsi(sdat3,j,memory)
        #print("RSI:",RSI)

        MFI = get_mfi(sdat3,j,memory)
        #print("MFI:",MFI)

        EMA = get_ema(sdat3,j,memory,EMAp)
        EMAp = EMA

        SO = get_so(sdat3,j,memory)

        N_close = sdat3[j+1,3]

        rec1 = [sdat3[j,2],sdat3[j,0],sdat3[j,1],sdat3[j,3],sdat3[j,4],RSI,MFI,EMA,SO,N_close]
        rec2 = [sdat3[j,3],RSI,MFI,EMA,SO,N_close]

        if(sdat3[j,4]!=0):
            d1 = {"Open":sdat3[j,2],"High":sdat3[j,0],"Low":sdat3[j,1],"Close":sdat3[j,3],"Volume":sdat3[j,4],"RSI":RSI,"MFI":MFI,"EMA":EMA,"SO":SO,"CloseNext":N_close}
            df1.loc[i] = rec1
            df2.loc[i] = rec2

    #ANN
    dataset = df1
    y = pd.DataFrame(dataset['CloseNext'])
    X = dataset.drop(['CloseNext'], axis = 1)

    X = np.array(X)
    y = np.array(y)
    #X = X[1700:2030,:]
    #y = y[1700:2030,:]
    y = y.flatten()

    #Feature scaling
    scaled = StandardScaler()
    scaled.fit(X)
    X = scaled.transform(X)

    #Train Test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

    ## ANN implementation
    # define base model of our neural network for regression taks
    def endgame():
        # Adding the neurons in various layers
        model = Sequential()
        model.add(Dense(9, input_dim=9, kernel_initializer='normal', activation='relu'))
        model.add(Dense(15, kernel_initializer='normal', activation='relu'))
        model.add(Dense(8, kernel_initializer='normal', activation='relu'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model for our use in KerasRegressor
        model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['mape'])
        return model

    ann_regression = KerasRegressor(build_fn = endgame, epochs=100, batch_size=5, verbose=1)

    ann_regression.fit(X_train,y_train)

    ann_predict = ann_regression.predict(X_test)

    error = mean_absolute_error(ann_predict,y_test)
    per_err = (error/np.mean(y_test)) * 100
    print('The mean absolute error is {} and percentage error is {}.'.format(error,per_err))
    
    return ann_regression.predict(scaled.transform([df1.iloc[-1,:-1].values.tolist()]))

In [2]:
#parameter
start_year = 2011
start_month = 1
start_day = 1
code = "RELIANCE.NS"

predicted_price  = predict_clossing_price(start_year, start_month, start_day, code)

<ipython-input-1-f078d159c202>:104: RuntimeWarning: invalid value encountered in float_scalars
  SO = ((sdata[m,3]-sdata[m,1])/(sdata[m,0]-sdata[m,1]))*100


Epoch 1/100
349/349 [==============================] - 1s 2ms/step - loss: 830613.9363 - mape: 99.2877
Epoch 2/100
349/349 [==============================] - 0s 1ms/step - loss: 261118.0811 - mape: 55.1514
Epoch 3/100
349/349 [==============================] - 0s 1ms/step - loss: 40779.1868 - mape: 21.0847
Epoch 4/100
349/349 [==============================] - 0s 1ms/step - loss: 31893.8507 - mape: 19.0618
Epoch 5/100
349/349 [==============================] - 0s 1ms/step - loss: 19776.5250 - mape: 16.0733
Epoch 6/100
349/349 [==============================] - 1s 1ms/step - loss: 13724.4165 - mape: 13.4543
Epoch 7/100
349/349 [==============================] - 0s 1ms/step - loss: 5591.2100 - mape: 8.4283
Epoch 8/100
349/349 [==============================] - 0s 1ms/step - loss: 2281.7259 - mape: 5.4138
Epoch 9/100
349/349 [==============================] - 0s 1ms/step - loss: 1389.1259 - mape: 3.7015
Epoch 10/100
349/349 [==============================] - 0s 1ms/step - loss: 720.1043 -

349/349 [==============================] - 0s 1ms/step - loss: 368.3814 - mape: 1.4833
Epoch 83/100
349/349 [==============================] - 0s 1ms/step - loss: 289.6273 - mape: 1.3904
Epoch 84/100
349/349 [==============================] - 0s 1ms/step - loss: 348.5240 - mape: 1.4881
Epoch 85/100
349/349 [==============================] - 0s 1ms/step - loss: 258.5772 - mape: 1.4193
Epoch 86/100
349/349 [==============================] - 0s 1ms/step - loss: 359.5942 - mape: 1.5372
Epoch 87/100
349/349 [==============================] - 0s 1ms/step - loss: 336.8454 - mape: 1.4475
Epoch 88/100
349/349 [==============================] - 0s 1ms/step - loss: 369.6389 - mape: 1.4592
Epoch 89/100
349/349 [==============================] - 0s 1ms/step - loss: 362.4246 - mape: 1.5096
Epoch 90/100
349/349 [==============================] - 0s 1ms/step - loss: 335.9230 - mape: 1.4270
Epoch 91/100
349/349 [==============================] - 0s 1ms/step - loss: 356.5862 - mape: 1.5087
Epoch 92/100


In [8]:
int(predicted_price)

2183